# Generate Sample Data
Creates sample DXF file from sample_objects.csv for the Laydown Planner

## 1. Install Dependencies

In [1]:
import subprocess
import sys

try:
    import ezdxf
    print('ezdxf already installed')
except ImportError:
    print('Installing ezdxf...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ezdxf'])
    import ezdxf
    print('ezdxf installed successfully')

ezdxf already installed


## 2. Load Objects from CSV

In [2]:
import pandas as pd
import os

# Load objects from CSV
objects_df = pd.read_csv('../data/sample_objects.csv')

print(f'Loaded {len(objects_df)} objects from sample_objects.csv')
print(f'\nColumns: {objects_df.columns.tolist()}')
print(f'\nFirst 5 objects:')
print(objects_df.head())

Loaded 12 objects from sample_objects.csv

Columns: ['object_id', 'name', 'category', 'width', 'length', 'height', 'weight', 'stackable', 'max_stack_height']

First 5 objects:
  object_id             name   category  width  length  height  weight  \
0    OBJ001      Cement Bags  Materials   1.00     1.0     1.5      50   
1    OBJ002      Steel Beams  Materials   0.50    12.0     0.5     500   
2    OBJ003    Wooden Planks  Materials   0.25     4.0     0.1      30   
3    OBJ004  Concrete Blocks  Materials   0.40     0.2     0.2      20   
4    OBJ005      Scaffolding  Equipment   1.50     1.5     0.2     100   

   stackable  max_stack_height  
0       True                 5  
1      False                 1  
2       True                 8  
3       True                 6  
4       True                 4  


## 3. Generate DXF from Objects

In [ ]:
import ezdxf
import pandas as pd
import os

# Load objects from CSV
objects_df = pd.read_csv('../data/sample_objects.csv')

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

print('Creating DXF with objects from sample_objects.csv...')
print(f'Processing {len(objects_df)} objects')

# Create a new DXF file
doc = ezdxf.new()
msp = doc.modelspace()

# Add site boundary
msp.add_lwpolyline([(0, 0), (200, 0), (200, 200), (0, 200), (0, 0)], close=True)

# Track position for layout
current_x = 5
current_y = 5
row_max_height = 0
max_row_width = 180

# Add objects as rectangles using LENGTH and WIDTH
for idx, row in objects_df.iterrows():
    # Get LENGTH and WIDTH from CSV
    obj_length = float(row['length'])
    obj_width = float(row['width'])
    obj_name = row['name']
    obj_weight = row['weight']
    
    # Check if we need to wrap to next row
    if current_x + obj_length > 5 + max_row_width:
        current_x = 5
        current_y += row_max_height + 3
        row_max_height = 0
    
    # Draw rectangle using actual length and width
    rect = msp.add_lwpolyline(
        [(current_x, current_y), 
         (current_x + obj_length, current_y), 
         (current_x + obj_length, current_y + obj_width), 
         (current_x, current_y + obj_width), 
         (current_x, current_y)],
        close=True
    )
    rect.dxf.color = 3  # Green
    
    # Add object name - centered in rectangle
    name_text = msp.add_text(
        obj_name,
        dxfattribs={
            'insert': (current_x + obj_length/2, current_y + obj_width/2 + 0.3),
            'height': 0.4,
            'halign': 1,  # Center
            'valign': 0,  # Bottom
        }
    )
    
    # Add weight below name
    weight_text = msp.add_text(
        f'{obj_weight}kg',
        dxfattribs={
            'insert': (current_x + obj_length/2, current_y + obj_width/2 - 0.3),
            'height': 0.3,
            'halign': 1,  # Center
            'valign': 0,  # Bottom
        }
    )
    
    # Update position
    current_x += obj_length + 2
    row_max_height = max(row_max_height, obj_width)

# Save the DXF file
dxf_path = '../data/sample_laydown.dxf'
doc.saveas(dxf_path)

print(f'\n✓ DXF file created: {dxf_path}')
print(f'  - Objects: {len(objects_df)} rectangles')
print(f'  - Each rectangle uses actual LENGTH x WIDTH from CSV')
print(f'  - Labels: Object name and weight (positioned within rectangles)')

Creating DXF with objects from sample_objects.csv...
Processing 12 objects

✓ DXF file created: ../data/sample_laydown.dxf
  - Objects: 12 rectangles
  - Each rectangle uses actual LENGTH x WIDTH from CSV
  - Labels: Object name and weight


## 4. Verify DXF File

In [5]:
import ezdxf
import os

dxf_path = '../data/sample_laydown.dxf'

if os.path.exists(dxf_path):
    doc = ezdxf.readfile(dxf_path)
    msp = doc.modelspace()
    
    print(f'✓ DXF file verified')
    print(f'  File size: {os.path.getsize(dxf_path)} bytes')
    print(f'  DXF Version: {doc.dxfversion}')
    print(f'  Total entities: {len(msp)}')
    
    # Count entity types
    entity_types = {}
    for entity in msp:
        etype = entity.dxftype()
        entity_types[etype] = entity_types.get(etype, 0) + 1
    
    print(f'\n  Entity types:')
    for etype, count in sorted(entity_types.items()):
        print(f'    {etype}: {count}')
else:
    print(f'❌ DXF file not found: {dxf_path}')

✓ DXF file verified
  File size: 24702 bytes
  DXF Version: AC1027
  Total entities: 37

  Entity types:
    LWPOLYLINE: 13
    TEXT: 24


## 5. Summary

In [6]:
print('\n=== Sample Data Generation Complete ===')
print('\nCreated files:')
print('  ✓ data/sample_laydown.dxf')
print('\nThe DXF contains:')
print('  - Site boundary rectangle')
print('  - Object bounding rectangles from sample_objects.csv')
print('  - Object labels and weights')
print('\nYou can now run example_analysis.ipynb to visualize the DXF!')


=== Sample Data Generation Complete ===

Created files:
  ✓ data/sample_laydown.dxf

The DXF contains:
  - Site boundary rectangle
  - Object bounding rectangles from sample_objects.csv
  - Object labels and weights

You can now run example_analysis.ipynb to visualize the DXF!
